In [1]:
import torch
import numpy as np
from mnist import MNIST
mndata = MNIST('./python-mnist/data')
images, labels = mndata.load_training()

In [2]:
def get_feature(x):

    feature=[0,0,0,0]
    xa = np.array(x)
    xt = torch.from_numpy(xa.reshape(28,28))
    feature=xt.view(1,784)
    feature=feature/2550.0
    return feature.cuda()
def one_hot(gt):
    gt_vector = torch.ones(1,10)
    gt_vector *= 0
    gt_vector[0,gt] = 1
    return gt_vector

In [3]:
def model(feature,parameters):
    y=-1
    h = feature.mm(parameters[0])+parameters[1]
    h1 = torch.sigmoid(h).mm(parameters[2])+parameters[3]
    y =torch.sigmoid(h1)
    #y = 1.0/(1.0+torch.exp(-1.*h))
    return y
def get_acc(image_data,image_label,parameters,start_i=0,end_i=0):

    correct=0
    if end_i==0:
        end_i=len(image_label)
    for i in range(start_i,end_i):
             #print(image_label[i])
             #y = model(get_feature(image_data[i]),weights)
             feature = get_feature(image_data[i])
             y = model(feature,parameters)
             #pdb.set_trace()
             gt = image_label[i]
             #pred=torch.argmin(torch.abs(y-gt)).item()
             #pred = torch.argmin(torch.from_numpy(np.array([torch.min((torch.abs(y-j))).item() for j in range(0,10)]))).item()
             #pred = torch.argmin(torch.min(torch.abs(y-1))).item()
             pred = torch.argmax(y).item()
             #print("图像[%s]得分类结果是:[%s]"%(gt,pred))
             if gt==pred:
                 correct+=1
    #print("acc=%s"%(float(correct/20.0)))
    return  float(correct/float(end_i-start_i))

In [4]:
from tqdm import tqdm 
import torch.nn as nn
entroy=nn.CrossEntropyLoss()
def train_model(train_image_data,train_image_label,test_image_data,test_image_label,parameters,lr):
    loss_value_before=1000000000000000.
    loss_value=10000000000000.
    for epoch in range(0,50):
        loss_value_before=loss_value
        loss_value=0
        print("epoch=%s"%(epoch))
        for i in tqdm(range(len(train_image_label))):
            #print(image_label[i])
            #y = model(get_feature(image_data[i]),weights)
            feature = get_feature(train_image_data[i])
            y = model(feature,parameters)
            gt = train_image_label[i]
            # 只关心一个值
            #loss = torch.sum((y[0,gt:gt+1]-gt).mul(y[0,gt:gt+1]-gt))
            gt_vector = one_hot(gt).cuda()
            #pdb.set_trace()
            # 关心所有值
            #loss = entroy(y,gt_vector.long())
            loss = torch.sum((y-gt_vector).mul(y-gt_vector))
            # 用log的方式
            #pdb.set_trace()
            #loss = -torch.log(y[0,gt])-torch.sum(torch.log(1.0-y[0,0:gt]))-torch.sum(torch.log(1-y[0,gt:-1]))
            # 优化loss，正样本接近1，负样本远离1
            #loss1 = (y-1.0).mul(y-1.0)
            #loss = loss1[0,gt]+torch.sum(1.0/(loss1[0,0:gt]))+torch.sum(1.0/(loss1[0,gt:-1]))
            #print("%s,%s"%(y[0,gt:gt+1],gt))
            
            #loss.data.add_(loss.data) 
            loss_value += loss.data.item()
            loss.data += loss.data
            #print("loss=%s"%(loss))
            #weights =
            # 更新公式
            # w  = w - (y-y1)*x*lr
            #feature=feature.view(6)
            #lr=-lr
            #weights[0,0] = weights[0,0]+ (y.item()-image_label[i])*feature[0]*lr
            #weights[1,0] = weights[1,0]+ (y.item()-image_label[i])*feature[1]*lr
            #weights[2,0] = weights[2,0]+ (y.item()-image_label[i])*feature[2]*lr
            #weights[3,0] = weights[3,0]+ (y.item()-image_label[i])*feature[3]*lr
            #weights[4,0] = weights[4,0]+ (y.item()-image_label[i])*feature[4]*lr
            #weights[5,0] = weights[5,0]+ (y.item()-image_label[i])*feature[5]*lr
            #weights[6,0] = weights[6,0]+ (y.item()-image_label[i])*lr
            if i%10==0:
                loss.backward()
                for para in parameters:
                    para.data.sub_(para.grad.data*lr)
                    para.grad.data.zero_()
                loss.data.zero_()
            #loss.data=
        #import pdb
        #print("epoch=%s,loss=%s/%s,weights=%s"%(epoch,loss_value,loss_value_before,(weights[:,0:2]).view(14)))
        train_acc=get_acc(train_image_data,train_image_label,parameters)
        test_acc =get_acc(test_image_data,test_image_label,parameters)
        print("epoch=%s,loss=%s/%s,train/test_acc:%s/%s"%(epoch,loss_value,loss_value_before,train_acc,test_acc))

    return parameters

In [5]:
weights0 = torch.randn(784,30,requires_grad = True).cuda()
bias0 = torch.zeros(1,30,requires_grad = True).cuda()
weights1= torch.randn(30,10,requires_grad = True).cuda()
bias1 = torch.zeros(1,10,requires_grad = True).cuda()
parameters=torch.nn.ParameterList([torch.nn.Parameter(weights0),torch.nn.Parameter(bias0),torch.nn.Parameter(weights1),torch.nn.Parameter(bias1)])
# hct66 dataset , 10 samples
mndata = MNIST('./python-mnist/data/')
train_image_data, train_image_label = mndata.load_training()
train_image_data=train_image_data[0:1000]
train_image_label=train_image_label[0:1000]
test_image_data, test_image_label = mndata.load_testing()
test_image_data=test_image_data[0:200]
test_image_label=test_image_label[0:200]
lr = float(3)
# 对模型进行训练：
parameters = train_model(train_image_data, train_image_label,test_image_data, test_image_label,parameters,lr)

 19%|█▉        | 193/1000 [00:00<00:00, 1929.82it/s]

epoch=0


 20%|██        | 202/1000 [00:00<00:00, 2017.31it/s]

epoch=0,loss=1294.2710938244709/10000000000000.0,train/test_acc:0.107/0.14
epoch=1


 20%|██        | 200/1000 [00:00<00:00, 1994.27it/s]

...

epoch=47,loss=512.5563843404933/512.4853030185293,train/test_acc:0.633/0.735
epoch=48


 20%|██        | 201/1000 [00:00<00:00, 2008.44it/s]

epoch=48,loss=512.626383357236/512.5563843404933,train/test_acc:0.633/0.735
epoch=49


100%|██████████| 1000/1000 [00:00<00:00, 2019.11it/s]


epoch=49,loss=512.6954121972585/512.626383357236,train/test_acc:0.633/0.735


In [7]:
#test
test_acc =get_acc(test_image_data,test_image_label,parameters)
print("test_acc:%s"%(test_acc))

test_acc:0.735
